# Building a Red-Black Tree

In this notebook, we'll walk through how you might build a red-black tree. Remember, we need to follow the red-black tree rules, on top of the binary search tree rules. Our new rules are:

* All nodes have a color
* All nodes have two children (use NULL nodes)
  * All NULL nodes are colored black
* If a node is red, its children must be black
* The root node must be black (optional)
  * We'll go ahead and implement without this for now
* Every path to its descendant NULL nodes must contain the same number of black nodes

### Sketch

Similar to our binary search tree implementation, we will define a class for nodes and a class for the tree itself. The `Node` class will need a couple new attributes. It is no longer enough to only know the children, because we need to ask questions during insertion like, "what color is my parent's sibling?". So we will add a parent link as well as the color.

In [4]:
class Node(object):
    def __init__(self, value, parent, color):
        self.value = value
        self.left = None
        self.right = None
        self.parent = parent
        self.color = color
        
    def __repr__(self):
        print_color = 'R' if self.color == 'red' else 'B'
        return '%d%s' % (self.value, print_color)

def grandparent(node):
    if node.parent == None:
        return None
    return node.parent.parent

# Helper for finding the node's parent's sibling
def pibling(node):
    p = node.parent
    gp = grandparent(node)
    if gp == None:
        return None
    if p == gp.left:
        return gp.right
    if p == gp.right:
        return gp.left

class RedBlackTree(object):
    def __init__(self, root):
        self.root = Node(root, None, 'red')
        
    def __iter__(self):
        yield from self.root.__iter__()
        
    def insert(self, new_val):
        new_node = self.insert_helper(self.root, new_val)
        self.rebalance(new_node)

    def insert_helper(self, current, new_val):
        if current.value < new_val:
            if current.right:
                return self.insert_helper(current.right, new_val)
            else:
                current.right = Node(new_val, current, 'red')
                return current.right
        else:
            if current.left:
                return self.insert_helper(current.left, new_val)
            else:
                current.left = Node(new_val, current, 'red')
                return current.left

    def rebalance(self, node):    
        # Case 1 : We have just inserted the root node
        if node.parent == None:
            return
        
        # Case 2 : We inserted under a black parent node
        # Thinking through this, we can observe the following: We inserted a red node beneath a black node. The new children (the NULL nodes) are black by definition, and our red node replaced a black NULL node. So the number of black nodes for any paths from parents is unchanged. Nothing to do in this case, either.
        if node.parent.color == 'black':
            return
        
        # Case 3 : The parent and its sibling of the newly inserted node are both red
        #  In this specific case, we can flip the color of the parent and its sibling. We know they're both red in this case, which means the grandparent is black. It will also need to flip. At that point we will have a freshly painted red node at the grandparent. At that point, we need to do the same evaluation! If the grandparent turns red, and its sibling is also red, that's case 3 again. Guess what that means! Time for more recursion.
        if pibling(node) and pibling(node).color == 'red':
            pibling(node).color = 'black'
            node.parent.color = 'black'
            return self.rebalance(grandparent(node))
        
        gp = grandparent(node)        
        # Small change, if there is no grandparent, cases 4 and 5
        # won't apply
        if gp == None:
            return
        
        # Case 4 : The newly inserted node has a red parent, but that parent has a black sibling
        # These last cases get more interesting. The criteria above actually govern case 4 and 5. What separates them is if the newly inserted node is on the inside or the outside of the sub tree. 

        if gp.left and node == gp.left.right:
            self.rotate_left(node.parent)
            node = node.left
        elif gp.right and node == gp.right.left:
            self.rotate_right(node.parent)
            node = node.right

        # Case 5 : If our new node is a left child of a left child, we rotate right. If our new node is a right of a right, we rotate left. This is done on the grandparent node.
        # But after this rotation, our colors will be off. Remember that for cases 3, 4, and 5, the parent of the new node is red. But we will have rotated a red node with a red child up, which violates our rule of all red nodes having two black children. So after rotating, we switch the colors of the (original) parent and grandparent nodes.
        p = node.parent
        gp = p.parent
        if node == p.left:
            self.rotate_right(gp)
        else:
            self.rotate_left(gp)
        p.color = 'black'
        gp.color = 'red'

    def rotate_left(self, node):
        # Save off the parent of the sub-tree we're rotating
        p = node.parent

        node_moving_up = node.right
        # After 'node' moves up, the right child will now be a left child
        node.right = node_moving_up.left

        # 'node' moves down, to being a left child
        node_moving_up.left = node
        node.parent = node_moving_up

        # Now we need to connect to the sub-tree's parent
        # 'node' may have been the root
        if p != None:
            if node == p.left:
                p.left = node_moving_up
            else:
                p.right = node_moving_up
        node_moving_up.parent = p

    def rotate_right(self, node):
        p = node.parent

        node_moving_up = node.left
        node.left = node_moving_up.right

        node_moving_up.right = node
        node.parent = node_moving_up

        # Now we need to connect to the sub-tree's parent
        if p != None:
            if node == p.left:
                p.left = node_moving_up
            else:
                p.right = node_moving_up
        node_moving_up.parent = p

### Testing

We've written a lot of code. Let's see how the tree mutates as we add nodes.

First, we'll need a way to visualize the tree. The below will nest, but remember the first child is always the left child.

In [5]:
def print_tree(node, level=0):
    print('   ' * (level - 1) + '+--' * (level > 0) + '%s' % node)
    if node.left:
        print_tree(node.left, level + 1)
    if node.right:
        print_tree(node.right, level + 1)

In [8]:
tree = RedBlackTree(9)
tree.insert(6)
tree.insert(19)

print_tree(tree.root)

9R
+--6R
+--19R


In [9]:
tree.insert(13)
print_tree(tree.root)

9R
+--6B
+--19B
   +--13R


Observe that 13 was inserted as red, and then because of Case 3, 6 and 19 flipped to black. 9 was also assigned red, but that was not a net change. Because we're not enforcing the optional "root is always black rule", this is acceptable.

Now let's cause some rotations. When we insert 16, it goes under 13, but 13 does not have a red sibling. 16 rotates into 13's spot, because it's currently an _inside_ sub-tree. Then 16 rotates into 19's spot. After these rotations, the ordering of the BST has been preserved _and_ our tree is balanced.

In [10]:
tree.insert(16)
print_tree(tree.root)

9R
+--6B
+--16B
   +--13R
   +--19R


# 👏👏👏👏👏

You've done it! Go ahead and pat yourself on the back! This is a complex use of a data structure that has significant power. It uses _O(n)_ space and insertions and search perform in _O(log n)_ time.

## Further Exercises

To continue exploring our red-black tree implementation, you might try the following.
* Observe that our current implementation will add duplicates of the same value. Is that desirable? How would you expect that to behave? Change the implementation to mark how many times the same value has been inserted.
* Implement search and see how it remains logarithmic for large data sets
* Tinker with the rotations and early escapes to see how they break (use `print_tree`)
* Consider adding deletion or sketching out how it should work